### AJUSTE DO AMBIENTE E IMPORTAÇÃO DE BIBLIOTECAS A SEREM USADAS

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import re
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, adjusted_rand_score
from sklearn.model_selection import train_test_split, GridSearchCV
# from skopt import BayesSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

### WEB SCRAPPING DOS DADOS DIRETO DO SITE 'CRUZEIROPEDIA'

In [3]:
# faz a requisição HTTP para a página principal
response = requests.get('https://cruzeiropedia.org/Categoria:Temporadas')

# faz o parsing do HTML da página principal
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
# encontra todas as tags <a> que contêm o nome das temporadas (identifica os links) 
temporadas = soup.find_all('a', {'title': 
                                 ['Categoria:Temporada 2011',
                                  'Categoria:Temporada 2012',
                                  'Categoria:Temporada 2013',
                                  'Categoria:Temporada 2014',
                                  'Categoria:Temporada 2015',
                                  'Categoria:Temporada 2016',
                                  'Categoria:Temporada 2017', 
                                  'Categoria:Temporada 2018', 
                                  'Categoria:Temporada 2019', 
                                  'Categoria:Temporada 2020', 
                                  'Categoria:Temporada 2021', 
                                  'Categoria:Temporada 2022',
                                  'Categoria:Temporada 2023',
                                  'Categoria:Temporada 2024']
                                  }
                                  )

In [6]:
# lista para armazenar os dados encontrados
anos = []
diasSemanas = []
datas = []
adversarios = []
pubPagantes = []
pubPresentes = []
rendaBrutas = []
estadios = []
campeonatos = []
placares = []
horarios = []

# 'varre' a lista de temporadas especificada acima
for temporada in temporadas:
    print(f"Começando: {temporada}")
    # obtém o link da temporada
    link_temporada = temporada.get('href')
    
    # faz a requisição HTTP para a página da temporada
    response_temporada = requests.get('https://cruzeiropedia.org' + link_temporada)
    
    # faz o parsing do HTML da página da temporada
    soup_temporada = BeautifulSoup(response_temporada.content, 'html.parser')
    
    # identifica os jogos que o cruzeiro foi mandante na temporada em questão
    jogos = soup_temporada.find_all('a', {'title': re.compile(r'(.+) (\d+)x(\d+) (.+) - (\d{2}/\d{2}/\d{4})')})

    # entra em cada um dos jogos
    for jogo in jogos:
        # obtém o link do jogo
        link_jogos = jogo.get('href')

        if "Slovan" in link_jogos:
            continue

        elif "Tr%C3%ADplice" in link_jogos:
            continue
    
        # faz a requisição HTTP para a página do jogo
        response_jogos = requests.get('https://cruzeiropedia.org' + link_jogos)
        # faz o parsing do html da página do jogo
        soup_jogos = BeautifulSoup(response_jogos.content, 'html.parser')
        # retira a informação do dia da semana e horário
        diaSemana = soup_jogos.find('b', string='Data:').next_sibling.strip()
        diasSemanas.append(diaSemana)

        ano = soup_jogos.find('b', string='Data:').find_next_sibling().find_next_sibling().text.strip()
        anos.append(ano)

        data = soup_jogos.find('b', string='Data:').find_next_sibling().text.strip()
        datas.append(data)

        adversario = soup_jogos.find('b', string='Placar').find_next_sibling().find_next_sibling().find_next_sibling().text.strip()
        adversarios.append(adversario)

        pubPagante = soup_jogos.find('b', string='Público pagante:').next_sibling.strip()
        pubPagantes.append(pubPagante)

        pubPresente = soup_jogos.find('b', string='Público Presente:').next_sibling.strip()
        pubPresentes.append(pubPresente)

        rendaBruta = soup_jogos.find('b', string='Renda Bruta:').next_sibling.strip()
        rendaBrutas.append(rendaBruta)

        estadio = soup_jogos.find('b', string='Estádio:').find_next_sibling().text.strip()
        estadios.append(estadio)
        
        campeonato = soup_jogos.find('div', class_='divFichaSessao')
        campeonato_element = campeonato.find('b')
        campeonato_element_title = campeonato_element.text.strip()
        campeonatos.append(campeonato_element_title)

        placar_element = soup_jogos.find('div', class_='divFicha')
        placar = placar_element.find('b').text
        placares.append(placar)

        horario = soup_jogos.find('b', string='Data:').find_next_sibling().find_next_sibling().next_sibling.strip()
        horarios.append(horario)



Começando: <a href="/Categoria:Temporada_2011" title="Categoria:Temporada 2011">Temporada 2011</a>
Começando: <a href="/Categoria:Temporada_2012" title="Categoria:Temporada 2012">Temporada 2012</a>
Começando: <a href="/Categoria:Temporada_2013" title="Categoria:Temporada 2013">Temporada 2013</a>
Começando: <a href="/Categoria:Temporada_2014" title="Categoria:Temporada 2014">Temporada 2014</a>
Começando: <a href="/Categoria:Temporada_2015" title="Categoria:Temporada 2015">Temporada 2015</a>
Começando: <a href="/Categoria:Temporada_2016" title="Categoria:Temporada 2016">Temporada 2016</a>
Começando: <a href="/Categoria:Temporada_2017" title="Categoria:Temporada 2017">Temporada 2017</a>
Começando: <a href="/Categoria:Temporada_2018" title="Categoria:Temporada 2018">Temporada 2018</a>
Começando: <a href="/Categoria:Temporada_2019" title="Categoria:Temporada 2019">Temporada 2019</a>
Começando: <a href="/Categoria:Temporada_2020" title="Categoria:Temporada 2020">Temporada 2020</a>
Começando:

### ESTRUTURAÇÃO DOS DADOS COLETADOS EM DATAFRAME + PRÉ-VISUALIZAÇÃO DOS DADOS

In [12]:
# ESTRUTURANDO OS DADOS CAPTURADOS EM UM DATAFRAME

df_raw = pd.DataFrame({'visitante':adversarios,
                       'ano': anos,
                       'dia da semana': diasSemanas,
                       'data': datas,
                       'publico presente': pubPresentes,
                       'publico pagante':pubPagantes,
                       'renda bruta': rendaBrutas,
                       'estadio':estadios,
                       'campeonato':campeonatos,
                       'placar':placares,
                       'horario':horarios})
df = df_raw.copy()

In [15]:
# Salvando em um excel para evitar de ter que rodar o web scrapping inteiro toda vez que for trabalhar 

df_raw.to_excel("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\0. Datasets Usados\\Dataset Cruzeiro\\Cruzeiro.xlsx", index=False)

In [44]:
df_raw = pd.read_csv("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\raw\\df_jogos_raw.csv")

df = df_raw.copy()

In [45]:
# Análise da cauda final do df para ver se o script deu certo e se são os jogos mais recentes mesmo
df.tail(5)

,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario
887,Cruzeiro,2024,"sábado,",23 de novembro,Não disponível,Não disponível,Não informado,La Nueva Olla,Jogo único - Final da Copa Sul-Americana 2024,3 × 1,às 17:00
888,Grêmio,2024,"quarta-feira,",27 de novembro,21.009,17.723 (4.416 sócios),"R$ 717.982,50",Mineirão,35ª rodada do Campeonato Brasileiro 2024,1 × 1,às 21:00
889,Cruzeiro,2024,"domingo,",1 de dezembro,6.133,6.133,"R$ 280.740,00",Nabi Abi Chedid,36ª rodada do Campeonato Brasileiro 2024,1 × 1,às 18:30
890,Palmeiras,2024,"quarta-feira,",4 de dezembro,0,0,"R$ 0,00",Mineirão,37ª rodada do Campeonato Brasileiro 2024,1 × 2,às 21:30
891,Cruzeiro,2024,"domingo,",8 de dezembro,13.518,13.518,"R$ 368.820,00",Alfredo Jaconi,38ª rodada do Campeonato Brasileiro 2024,0 × 1,às 16:00


### FILTRANDO APENAS CAMPEONATO BRASILEIRO - AMOSTRA A SER TRABALHADA

In [48]:
df['dia da semana'].value_counts()

dia da semana
domingo,          329
quarta-feira,     226
sábado,           170
quinta-feira,      76
terça-feira,       47
sexta-feira,       26
segunda-feira,     18
Name: count, dtype: int64

In [ ]:
# AJUSTANDO OS NOMES DOS CAMPEONATOS
camp_ajustado = []
for i in range(0,len(df)):
    if "Mineiro" in df['campeonato'][i]:
        camp_ajustado.append('Campeonato Mineiro')
    elif "Libertadores" in df['campeonato'][i]:
        camp_ajustado.append('Libertadores')
    elif "Copa do Brasil" in df['campeonato'][i]:
        camp_ajustado.append('Copa do Brasil')
    elif "Brasileiro" in df['campeonato'][i]:
        camp_ajustado.append('Brasileirão')
    elif "Primeira Liga" in df['campeonato'][i]:
        camp_ajustado.append('Primeira Liga')
    elif "Sul-Americana" in df['campeonato'][i]:
        camp_ajustado.append('Sul-Americana')
    elif "Inconfidência" in df['campeonato'][i]:
        camp_ajustado.append('Inconfidência')
    elif 'Sul-Minas-Rio' in df['campeonato'][i]:
        camp_ajustado.append('Copa Sul-Minas-Rio')
    elif "Amistoso" in df['campeonato'][i]:
        camp_ajustado.append('Amistoso')
    else:
        camp_ajustado.append('nao')

df['camp_ajustado'] = camp_ajustado

In [35]:
# CONFERINDO OS NOMES DOS CAMPEONATOS - HÁ ERROS NOS AMISTOSOS
for i in df['camp_ajustado'].unique():
    display(df[df['camp_ajustado'] == i])

,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario,camp_ajustado
0,Cruzeiro,2011,"domingo,",23 de janeiro,Não disponível,1.243,"R$ 12.820,00",Parque do Sabiá,Amistoso 2011,0 × 3,às 17:00,Amistoso
62,América-MG,2012,"domingo,",22 de janeiro,2.170,2.170,"R$ 22.475,00",Parque do Sabiá,Amistosos 2012,2 × 3,às 17:00,Amistoso
63,Cruzeiro,2012,"sábado,",28 de janeiro,12.000,10.000,Não informado,Arena do Sapo,Amistoso 2012,1 × 2,às 16:00,Amistoso
120,Cruzeiro,2013,"domingo,",27 de janeiro,Não disponível,5.000,Não informado,Arena do Sapo,Amistoso,1 × 4,às 17:00,Amistoso
144,Cruzeiro,2013,"domingo,",23 de junho,Não disponível,5.724,Não informado,Lockhart,Amistoso 2013,0 × 4,às 20:30,Amistoso
145,Cruzeiro,2013,"sábado,",29 de junho,Não disponível,2.500,Não informado,Toyota Park,Amistoso 2013,1 × 2,às 17:00,Amistoso
217,Cruzeiro,2014,"domingo,",22 de junho,Não disponível,5.000,Não informado,Bowditch Stadium,Amistosos 2014,1 × 5,às 16:00,Amistoso
218,Cruzeiro,2014,"terça-feira,",24 de junho,Não disponível,Não disponível,Não informado,Veterans Memorial Stadium,Amistoso 2014,1 × 2,às 20:30,Amistoso
219,Cruzeiro,2014,"sexta-feira,",27 de junho,Não disponível,Não disponível,Não informado,Cotton Bowl,Amistoso 2014,3 × 5,às 21:30,Amistoso
220,Cruzeiro,2014,"quinta-feira,",3 de julho,Não disponível,Não disponível,Não informado,BBVA Compass Stadium,Amistoso 2014,0 × 2,às 22:00,Amistoso


,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario,camp_ajustado
1,Caldense,2011,"domingo,",30 de janeiro,Não disponível,4.151,"R$ 82.980,00",Arena do Jacaré,1ª rodada do Campeonato Mineiro 2011,3 × 0,às 17:00,Campeonato Mineiro
2,Cruzeiro,2011,"domingo,",6 de fevereiro,Não disponível,4.144,"R$ 55.230,00",Castor Cifuentes,2ª rodada do Campeonato Mineiro 2011,0 × 1,às 19:30,Campeonato Mineiro
3,Atlético-MG,2011,"sábado,",12 de fevereiro,Não disponível,9.793,"R$ 267.256,88",Arena do Jacaré,3º rodada do Campeonato Mineiro 2011,3 × 4,às 19:30,Campeonato Mineiro
5,Ipatinga,2011,"sábado,",19 de fevereiro,2.045,1.664,"R$ 25.108,13",Arena do Jacaré,4ª rodada do Campeonato Mineiro 2011,2 × 0,às 17:00,Campeonato Mineiro
7,Cruzeiro,2011,"sábado,",26 de fevereiro,Não disponível,5.000,"R$ 111.350,00",Arena do Dragão,5ª rodada da Campeonato Mineiro 2011,1 × 2,às 16:00,Campeonato Mineiro
...,...,...,...,...,...,...,...,...,...,...,...,...
836,Uberlândia,2024,"sábado,",2 de março,32.868,29.712 (1.567 sócios),"R$ 1.404.071,00",Mineirão,8ª rodada do Campeonato Mineiro 2024,2 × 0,às 16:30,Campeonato Mineiro
837,Cruzeiro,2024,"domingo,",10 de março,Não disponível,Não disponível,Não informado,Ipatingão,Jogo de ida - Semifinal do Campeonato Mineiro ...,0 × 0,às 19:30,Campeonato Mineiro
838,Tombense,2024,"sábado,",16 de março,40.158,35.980 (1.790 sócios),"R$ 1.689.992,50",Mineirão,Jogo de volta - Semifinal do Campeonato Mineir...,3 × 1,às 16:30,Campeonato Mineiro
839,Cruzeiro,2024,"sábado,",30 de março,42.592,38.078,"R$ 3.037.109,32",Arena MRV,Jogo de ida - Final do Campeonato Mineiro 2024,2 × 2,às 16:30,Campeonato Mineiro


,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario,camp_ajustado
4,Estudiantes,2011,"quarta-feira,",16 de fevereiro,Não disponível,10.955,"R$ 377.267,50",Arena do Jacaré,1ª Rodada - Fase de Grupos da Copa Libertadore...,5 × 0,às 22:00,Libertadores
6,Guarani-PAR,2011,"terça-feira,",22 de fevereiro,13.271,12.067,"R$ 275.668,02",Arena do Jacaré,2ª rodada - Fase de Grupos da Copa Libertadore...,4 × 0,às 19:15,Libertadores
8,Cruzeiro,2011,"quarta-feira,",2 de março,Não disponível,Não disponível,Não informado,Manuel Toro,3ª rodada - Fase de Grupos da Copa Libertadore...,0 × 0,às 19:30,Libertadores
11,Tolima,2011,"quarta-feira,",16 de março,Não disponível,8.198,"R$ 187.000,00",Arena do Jacaré,4ª rodada - Fase de Grupos da Copa Libertadore...,6 × 1,às 21:50,Libertadores
14,Cruzeiro,2011,"quarta-feira,",30 de março,Não disponível,Não disponível,Não informado,Defensores del Chaco,5ª rodada - Fase de Grupo da Copa Libertadores...,0 × 2,às 22:00,Libertadores
16,Cruzeiro,2011,"quarta-feira,",13 de abril,Não disponível,Não disponível,Não informado,Ciudad de La Plata,6ª rodada - Fase de Grupos da Copa Libertadore...,0 × 3,às 21:50,Libertadores
17,Cruzeiro,2011,"quarta-feira,",27 de abril,Não disponível,Não disponível,Não informado,Palo Grande,Jogo de ida - Oitavas de final da Copa Liberta...,1 × 2,às 21:50,Libertadores
21,Once Caldas,2011,"quarta-feira,",4 de maio,15.962,14.972,"R$ 352.780,39",Arena do Jacaré,Jogo de volta - Oitavas de final da Copa Liber...,0 × 2,às 21:50,Libertadores
187,Cruzeiro,2014,"quarta-feira,",12 de fevereiro,Não disponível,Não disponível,Não informado,Huancayo,1ª rodada - Fase de Grupos da Copa Libertadore...,2 × 1,às 22:00,Libertadores
191,Universidad de Chile,2014,"terça-feira,",25 de fevereiro,29.120,27.757,"R$ 957.725,00",Mineirão,2ª rodada - Fase de Grupos da Copa Libertadore...,5 × 1,às 17:30,Libertadores


,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario,camp_ajustado
24,Cruzeiro,2011,"domingo,",22 de maio,Não disponível,10.231,"R$ 120.670,00",Orlando Scarpelli,1ª rodada do Campeonato Brasileiro 2011,1 × 0,às 16:00,Brasileirão
25,Palmeiras,2011,"domingo,",29 de maio,Não disponível,9.080 (865 sócios),"R$ 147.838,25",Arena do Jacaré,2ª rodada do Campeonato Brasileiro 2011,1 × 1,às 16:00,Brasileirão
26,Cruzeiro,2011,"domingo,",5 de junho,Não disponível,4.530,"R$ 131.365,00",Engenhão,3ª rodada do Campeonato Brasileiro 2011,2 × 1,às 16:00,Brasileirão
27,Santos,2011,"sábado,",11 de junho,7.133,6.073 (668 sócios),"R$ 104.820,00",Arena do Jacaré,4ª rodada do Campeonato Brasileiro 2011,1 × 1,às 18:30,Brasileirão
28,Cruzeiro,2011,"sábado,",18 de junho,5.627,5.027,"R$ 87.145,00",Arena do Jacaré,5ª rodada do Campeonato Brasileiro 2011,1 × 1,às 21:00,Brasileirão
...,...,...,...,...,...,...,...,...,...,...,...,...
886,Cruzeiro,2024,"quarta-feira,",20 de novembro,Não disponível,Não disponível,Não informado,Arena Corinthians,34ª rodada do Campeonato Brasileiro 2024,2 × 1,às 11:00,Brasileirão
888,Grêmio,2024,"quarta-feira,",27 de novembro,21.009,17.723 (4.416 sócios),"R$ 717.982,50",Mineirão,35ª rodada do Campeonato Brasileiro 2024,1 × 1,às 21:00,Brasileirão
889,Cruzeiro,2024,"domingo,",1 de dezembro,6.133,6.133,"R$ 280.740,00",Nabi Abi Chedid,36ª rodada do Campeonato Brasileiro 2024,1 × 1,às 18:30,Brasileirão
890,Palmeiras,2024,"quarta-feira,",4 de dezembro,0,0,"R$ 0,00",Mineirão,37ª rodada do Campeonato Brasileiro 2024,1 × 2,às 21:30,Brasileirão


,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario,camp_ajustado
69,Cruzeiro,2012,"quarta-feira,",7 de março,Não disponível,5.174,"R$ 113.420,00",Arena da Floresta,1ª fase - Copa do Brasil 2012,0 × 6,às 22:00,Copa do Brasil
75,Cruzeiro,2012,"quarta-feira,",11 de abril,Não disponível,7.085,"R$ 102.760,00",Arena Condá,Jogo de ida - 2ª fase da Copa do Brasil 2012,1 × 1,às 22:00,Copa do Brasil
77,Chapecoense,2012,"quarta-feira,",18 de abril,5.311,4.078,"R$ 66.990,00",Arena do Jacaré,Jogo de volta - 2ª fase da Copa do Brasil 2012,4 × 1,às 21:50,Copa do Brasil
80,Cruzeiro,2012,"quarta-feira,",2 de maio,Não disponível,6.581,"R$ 92.360,00",Durival de Britto,Jogo de ida - Oitavas de Final da Copa do Bras...,1 × 0,às 21:50,Copa do Brasil
81,Atlético-PR,2012,"quarta-feira,",9 de maio,Não disponível,8.769,"R$ 156.707,00",Arena do Jacaré,Jogo de volta - Oitavas de final da Copa do Br...,1 × 2,às 22:00,Copa do Brasil
...,...,...,...,...,...,...,...,...,...,...,...,...
786,Cruzeiro,2023,"quinta-feira,",13 de abril,6.435,4.523,"R$ 132.246,50",Aflitos,Jogo de ida - 3ª fase da Copa do Brasil 2023,1 × 0,às 19:00,Copa do Brasil
789,Náutico,2023,"terça-feira,",25 de abril,21.657,19.489 (1.396 sócios),"R$ 554.127,00",Independência,Jogo de volta - 3ª fase da Copa do Brasil 2023,2 × 0,às 19:00,Copa do Brasil
794,Cruzeiro,2023,"quarta-feira,",17 de maio,27.522,27.522,"R$ 1.584.033,00",Arena Grêmio,Jogo de ida - Oitavas de final da Copa do Bras...,1 × 1,às 19:30,Copa do Brasil
797,Grêmio,2023,"quarta-feira,",31 de maio,41.090,37.310 (1.960 sócios),"R$ 1.961.605,00",Mineirão,Jogo de volta - Oitavas de Final da Copa do Br...,0 × 1,às 20:00,Copa do Brasil


,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario,camp_ajustado
326,Cruzeiro,2016,"quarta-feira,",27 de janeiro,7.040,Não disponível,"R$ 60.845,00",Heriberto Hülse,1ª rodada - 1ª fase da Copa Sul-Minas-Rio 2016...,1 × 1,às 19:30,Copa Sul-Minas-Rio
330,Fluminense,2016,"quarta-feira,",17 de fevereiro,Não disponível,21.118,"R$ 400.748,00",Mineirão,2ª rodada - 1ª fase da Copa Sul-Minas-Rio 2016...,3 × 4,às 19:30,Copa Sul-Minas-Rio
334,Atlético-PR,2016,"quarta-feira,",9 de março,4.476,4.476 (3.735 sócios),"R$ 79.211,00",Mineirão,3ª rodada - 1ª fase da Copa Sul-Minas-Rio 201...,2 × 1,às 19:30,Copa Sul-Minas-Rio


,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario,camp_ajustado
393,Atlético-MG,2017,"quarta-feira,",1 de fevereiro,41.430,39.794 (13.444 sócios),"R$ 1.139.052,00",Mineirão,1ª rodada - 1ª fase da Copa Primeira Liga 2017,1 × 0,às 19:30,Primeira Liga
395,Chapecoense,2017,"quinta-feira,",9 de fevereiro,7.057,4.823,"R$ 98.177,00",Mineirão,2ª rodada da Copa Primeira Liga 2017,2 × 0,às 21:45,Primeira Liga
406,Cruzeiro,2017,"terça-feira,",21 de março,Não disponível,2.238,"R$ 20.100,00",Arena Joinville,3ª rodada - 1ª fase da Copa Primeira Liga 2017,0 × 0,às 19:00,Primeira Liga
446,Grêmio,2017,"quarta-feira,",30 de agosto,7.545,5.680,"R$ 69.343,00",Mineirão,Quartas de final - Jogo único da Copa Primeira...,2 × 0,às 21:45,Primeira Liga
447,Cruzeiro,2017,"domingo,",3 de setembro,17.061,15.420,"R$ 150.183,00",Estádio do Café,Jogo único - Semi-Final da Copa Primeira Liga ...,2 × 2,às 11:00,Primeira Liga


,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario,camp_ajustado
409,Nacional-PAR,2017,"terça-feira,",4 de abril,16.784,13.747,"R$ 247.108,00",Mineirão,Jogo de ida - 1ª fase da Copa Sul-Americana 2017,2 × 1,às 21:45,Sul-Americana
418,Cruzeiro,2017,"quarta-feira,",10 de maio,Não disponível,Não disponível,Não informado,Defensores del Chaco,Jogo da volta - 1ª fase da Copa Sul-Americana ...,2 × 1,às 19:15,Sul-Americana
840,Cruzeiro,2024,"quinta-feira,",4 de abril,3.092,3.092,Não informado,Olímpico Atahualpa,1ª rodada - Fase de Grupos da Copa Sul-America...,0 × 0,às 21:00,Sul-Americana
842,Alianza FC,2024,"quinta-feira,",11 de abril,18.818,16.936,"R$ 470.170,00",Mineirão,2ª rodada - Fase de Grupos da Copa Sul-America...,3 × 3,às 21:00,Sul-Americana
846,Cruzeiro,2024,"terça-feira,",23 de abril,1.182,Não disponível,Não informado,Estadio Regional,3ª rodada - Fase de Grupos da Copa Sul-America...,0 × 0,às 19:00,Sul-Americana
848,Cruzeiro,2024,"terça-feira,",7 de maio,3.068,Não disponível,Não informado,Armando Maestre Pavajeau,4ª rodada - Fase de Grupos da Copa Sul-America...,0 × 3,às 21:30,Sul-Americana
850,Unión La Calera,2024,"quinta-feira,",16 de maio,17.712,15.940,"R$ 720.682,50",Independência,5ª rodada - Fase de Grupos da Copa Sul-America...,1 × 0,às 21:00,Sul-Americana
851,Universidad de Quito,2024,"quinta-feira,",30 de maio,55.254,46.965,"R$ 2.770.277,50",Mineirão,6ª rodada - Fase de Grupos da Copa Sul-America...,1 × 0,às 21:00,Sul-Americana
868,Cruzeiro,2024,"quinta-feira,",15 de agosto,43.973,43.973,Não informado,La Bombonera,Jogo de ida - Oitavas de Final da Copa Sul-Ame...,1 × 0,às 21:30,Sul-Americana
870,Boca Juniors,2024,"quinta-feira,",22 de agosto,58.323,58.323 (43.742 sócios),"R$ 4.596.569,00",Mineirão,Jogo de volta - Oitavas de Final da Copa Sul-A...,2 × 1,às 21:30,Sul-Americana


,visitante,ano,dia da semana,data,publico presente,publico pagante,renda bruta,estadio,campeonato,placar,horario,camp_ajustado
622,Patrocinense,2020,"sábado,",1 de agosto,0,0,"R$ 0,00",Mineirão,Jogo único - Semifinal do Troféu Inconfidência...,3 × 0,às 14:30,Inconfidência


In [21]:
# AJUSTANDO MANUALMENTE OS ERROS NOS AMISTOSOS
df.loc[159, 'camp_ajustado'] = 'Copa-Sul-Minas-Rio'
df.loc[161, 'camp_ajustado'] = 'Copa-Sul-Minas-Rio'

In [ ]:
df.loc[326,'campeonato']

'1ª rodada - 1ª fase da Copa Sul-Minas-Rio 2016  Torneio Amistoso 2016'

In [22]:
# FILTRANDO APENAS BRASILEIRAO
df = df[df['camp_ajustado'] == "Brasileirão"].reset_index(drop='index')

### DATA WRANGLING

In [23]:
# RETIRADA DA VÍRGULA NO FINAL DO DIA DA SEMANA
df['dia da semana'] = df['dia da semana'].str[:-1]

# SEPARAÇÃO DA COLUNA DATA EM DIA E MÊS
df[['dia do mes', 'mes']] = df['data'].str.split(' de ', expand= True)

# RETIRADA DO R$ NO RENDA BRUTA
df['renda bruta'] = df['renda bruta'].str.slice(start=2)

#RETIRADA DOS PARÊNTESES COM AS INFOS DOS SÓCIOS DO PUB PAGANTE 
df[['publico pagante','erro1','erro2']] = df['publico pagante'].str.split(' ',expand=True)

# AJUSTE DA COLUNA DE HORÁRIO - RETIRADA DA STRING ÀS E DOS :/;
df['horario'] = df['horario'].str.slice(start=3)
df['horario'] = df['horario'].str.replace(';',':')
df['horario'] = df['horario'].str.replace(':','')

#RETIRANDO AS COLUNAS CRIADAS APENAS PARA RETIRADA DOS SÓCIOS
df.drop(columns=['erro1','erro2'],inplace=True)

# CONSERTANDO OS SEPARADORES NÚMERICOS (, E .) DAS COLUNAS NUMÉRICAS
for i in range(0,len(df)):
    df['renda bruta'][i] = df['renda bruta'][i].replace('.','')
    df['renda bruta'][i] = df['renda bruta'][i].replace(',','.')
    df['publico presente'][i] = df['publico presente'][i].replace('.','')
    df['publico pagante'][i] = df['publico pagante'][i].replace('.','')

# AJUSTANDO A COLUNA DE PLACAR E CRIANDO UMA NOVA PARA ANLALISAR VITÓRIA
df[['golsMandante','golsVisitante']] = df['placar'].str.split(' × ',expand=True)
df['pontosAlcancados'] = None
for i in range(0,len(df)):
    if df['golsMandante'][i] == df['golsVisitante'][i]:
        df['pontosAlcancados'][i] = 1

    elif df['golsMandante'][i] > df['golsVisitante'][i] and df['visitante'][i] == 'Cruzeiro':
        df['pontosAlcancados'][i] = 0

    elif df['golsMandante'][i] > df['golsVisitante'][i] and df['visitante'][i] != 'Cruzeiro':
        df['pontosAlcancados'][i] = 3

    elif df['golsMandante'][i] < df['golsVisitante'][i] and df['visitante'][i] == 'Cruzeiro':
        df['pontosAlcancados'][i] = 3

    elif df['golsMandante'][i] < df['golsVisitante'][i] and df['visitante'][i] != 'Cruzeiro':
        df['pontosAlcancados'][i] = 0

# CRIANDO UMA NOVA COLUNA COM O RESULTADO DO JOGO
df['resultado'] = None
for i in range(0,len(df)):
    if df['pontosAlcancados'][i] == 0:
        df['resultado'][i] = "Derrota"
    elif df['pontosAlcancados'][i] == 1:
        df['resultado'][i] = "Empate"
    elif df['pontosAlcancados'][i] == 3:
        df['resultado'][i] = "Vitoria"
    else:
        df['resultado'][i] = "Erro"

# CRIANDO NOVA COLUNA DE APROVEITAMENTO

df['aproveitamento_geral_previo'] = None

indice_primeiro_jogo_do_ano = 0
ano_em_questao = None

for i,v in enumerate(range(0,len(df))):
    if i == 0:
        df['aproveitamento_geral_previo'][i] = 0
        ano_em_questao = df['ano'][i]  

    elif df['ano'][i] != df['ano'][i-1]:
        df['aproveitamento_geral_previo'][i] = 0
        indice_primeiro_jogo_do_ano = i
        ano_em_questao = df['ano'][i]

    else:
        df['aproveitamento_geral_previo'][i] = df[(df.index < i)&(df['ano'] == ano_em_questao)]['pontosAlcancados'].sum()/((i-indice_primeiro_jogo_do_ano)*3)

# CRIANDO COLUNA DE JOGOS DE INVENCIBILIDADE

df['sequencia_invencibilidade_previa'] = None
seq_teste = 0
for i,v in enumerate(range(0,len(df))):
    if i == 0:
        df['sequencia_invencibilidade_previa'][i] = 0
    elif df['ano'][i] != df['ano'][i-1]:
        df['sequencia_invencibilidade_previa'][i] = 0
    elif df['resultado'][i-1] == 'Vitoria':
        seq_teste += 1
        df['sequencia_invencibilidade_previa'][i] = seq_teste
    elif df['resultado'][i-1] == 'Empate':
        seq_teste += 1
        df['sequencia_invencibilidade_previa'][i] = seq_teste
    else:
        seq_teste = 0
        df['sequencia_invencibilidade_previa'][i] = 0

# CRINDO UMA VARIÁVEL DUMMY INDICANDO SE TRATA DO PRIMEIRO JOGO DA TEMPORADA
df['estreia'] = None
for i,v in enumerate(range(0,len(df))):
    if i == 0:
        df['estreia'][i] = 1
    elif df['ano'][i] != df['ano'][i-1]:
        df['estreia'][i] = 1
    else:
        df['estreia'][i] = 0

# CRINDO UMA VARIÁVEL DUMMY INDICANDO SE É CLASSICO
df['classico'] = None
for i,v in enumerate(range(0,len(df))):
    if df['visitante'][i] == "Atlético-MG":
        df['classico'][i] = 1
    else:
        df['classico'][i] = 0

### DATA CLEANING

In [24]:
# CONFERINDO LINHAS DUPLICADAS
df.duplicated().sum()

0

In [25]:
# CONFERINDO LINHAS NULAS
df.isnull().sum()

visitante                           0
ano                                 0
dia da semana                       0
data                                0
publico presente                    0
publico pagante                     0
renda bruta                         0
estadio                             0
campeonato                          0
placar                              0
horario                             0
camp_ajustado                       0
dia do mes                          0
mes                                 0
golsMandante                        0
golsVisitante                       0
pontosAlcancados                    0
resultado                           0
aproveitamento_geral_previo         0
sequencia_invencibilidade_previa    0
estreia                             0
classico                            0
dtype: int64

In [26]:
# CONFERINDO FORMATOS DE ESCRITA DISTINTOS NA COLUNA DE HORARIO
df['horario'].unique()

array(['1600', '1830', '2100', '1930', '2200', '1800', '2030', '2150',
       '1700', '1900', '1620', '15h', '1100', '2145', '2000', '1630',
       '1915', '2130', '2015', '1815', '190', '19h', '16h'], dtype=object)

In [27]:
# FORMATANDO CADA UM DOS REGISTROS INCOERENTES
for i,v in enumerate(df['horario']):
    if v == '190' or v == '19h':
        df['horario'][i] = '1900'
    if v == '16h':
        df['horario'][i] = '1600'
    if v == '15h':
        df['horario'][i] = '1500'
    else:
        continue

In [28]:
# NOVOS VALORES DE HORARIOS
df['horario'].unique()

array(['1600', '1830', '2100', '1930', '2200', '1800', '2030', '2150',
       '1700', '1900', '1620', '1500', '1100', '2145', '2000', '1630',
       '1915', '2130', '2015', '1815'], dtype=object)

In [29]:
# COLOCANDO O NOME ANTIGO E O NOVO NOME DA ARENA JACARÉ DA MESMA FORMA. TRATAM-SE DO MESMO ESTÁDIO

df['estadio'] = df['estadio'].replace('Arena Buser','Arena do Jacaré')

EXPLICAÇÃO DO REMOÇÃO DE ALGUMAS COLUNAS 

>>PUBLICO PRESENTE -> dados muito poluídos, com alguns registros de "não disponível".  
>>CAMPEONATO e CAMP_AJUSTADO -> como trata-se apenas de brasileirão não faz sentido mantê-las  
>>PLACAR -> a coluna placar foi segregada entre as colunas de golsCruzeiro e golsAdversario  
>>DATA -> coluna foi desmembrada e não faz mais sentido mantê-la  
>>ANO -> usada apenas como regra para novas variáveis. Sem usabilidade e sem sentido  
>>golsMandante, golsVisitante, pontosAlcancados e resultados -> informações usadas apenas para criar regras de novas colunas criadas. São informações que não são conhecidas no momento da predição e por isso não servem como variáveis x  
>>RENDA BRUTA -> vazaria dados. 

In [30]:
# RETIRANDO COLUNAS NÃO UTILIZADAS

colunas_excluidas = ['publico presente','placar','data','campeonato','camp_ajustado','golsMandante','golsVisitante','pontosAlcancados','resultado','renda bruta']

df = df.drop(columns=colunas_excluidas)

### DATA PREPARATION

In [31]:
# MANTENDO APENAS OS REGISTROS DO CRUZEIRO COMO MANDANTE
df = df[df['visitante'] != "Cruzeiro"].reset_index(drop='index')

In [32]:
# ALTERANDO OS NOMES DOS MESES PARA NÚMEROS
meses_para_numeros ={
                     'janeiro': 1,
                     'fevereiro': 2,
                     'março': 3,
                     'abril': 4,
                     'maio': 5,
                     'junho': 6,
                     'julho': 7,
                     'agosto': 8,
                     'setembro': 9,
                     'outubro': 10,
                     'novembro': 11,
                     'dezembro': 12
                    }

df['mes'] = df['mes'].map(meses_para_numeros)

In [33]:
# CONFERINDO TIPO DE DADOS PARA MANUTENÇÃO
df.dtypes

visitante                           object
ano                                  int64
dia da semana                       object
publico pagante                     object
estadio                             object
horario                             object
dia do mes                          object
mes                                  int64
aproveitamento_geral_previo         object
sequencia_invencibilidade_previa    object
estreia                             object
classico                            object
dtype: object

In [34]:
# ALTERANDO O TIPO DE DADOS DE CADA COLUNA UMA A UMA PARA FACILITAR A IDENTIFICAÇÃO

df['visitante'] = df['visitante'].astype('category')                         
df['dia da semana'] = df['dia da semana'].astype('category')                                                  
df['publico pagante']= df['publico pagante'].astype('float')                               
df['estadio'] = df['estadio'].astype('category')                                                             
df['dia do mes'] = df['dia do mes'].astype(int)                   
df['mes'] = df['mes'].astype(int)
df['ano'] = df['ano'].astype(int)                                                           
df['aproveitamento_geral_previo'] = df['aproveitamento_geral_previo'].astype('float')               
df['sequencia_invencibilidade_previa'] = df['sequencia_invencibilidade_previa'].astype(int)    
df['estreia'] = df['estreia'].astype(int)
df['classico'] = df['classico'].astype(int)
df['horario'] = df['horario'].astype(int)

In [35]:
# NOVOS TIPOS DE DADOS
df.dtypes

visitante                           category
ano                                    int32
dia da semana                       category
publico pagante                      float64
estadio                             category
horario                                int32
dia do mes                             int32
mes                                    int32
aproveitamento_geral_previo          float64
sequencia_invencibilidade_previa       int32
estreia                                int32
classico                               int32
dtype: object

In [36]:
df.tail()

,visitante,ano,dia da semana,publico pagante,estadio,horario,dia do mes,mes,aproveitamento_geral_previo,sequencia_invencibilidade_previa,estreia,classico
244,Vasco,2023,quarta-feira,0.0,Mineirão,1900,22,11,0.404040,1,0,0
245,Athletico,2023,quinta-feira,33805.0,Mineirão,2000,30,11,0.419048,3,0,0
246,Palmeiras,2023,quarta-feira,44190.0,Mineirão,2130,6,12,0.414414,5,0,0
247,Botafogo,2024,domingo,17283.0,Mineirão,1700,14,4,0.000000,0,1,0
248,Vitória,2024,domingo,16996.0,Mineirão,1600,26,4,0.444444,0,0,0


### MODELLING

#### TREINAMENTO INICIAL

In [37]:
# drop do jogo do vasco - será usado para predição futura por conta da restrição de público que houve no jogo
df.drop(244,inplace=True)

In [38]:
# RETIRADA DE ALGUMAS FEATURES QUE ESTAVAM PIORANDO A PERFORMANCE DO MODELO
df_predict = df.drop(['visitante','dia do mes','estreia'],axis=1)
df_final = pd.get_dummies(df_predict)

In [39]:
# SEPARAÇÃO DA VARIÁVEL TARGET E VARIÁVEIS PREDITORAS
y = df_final['publico pagante']
X = df_final.drop('publico pagante',axis=1)

In [40]:
# SPLIT DOS DADOS EM BASE DE TREINO E TESTE
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

# TREINAMENTO DO MODELO XGBOOST
xgb = XGBRegressor(n_estimators = 400, max_depth=25, random_state=200).fit(X_train,y_train)

# PREDIÇÃO DO MODELO XGBOOST
yxgb = xgb.predict(X_test)

# MÉTRICAS DO MODELO XGBOOST
mae_xgb = mean_absolute_error(y_test,yxgb)
mse_xgb = mean_squared_error(y_test,yxgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = r2_score(y_test,yxgb)
adj_r2_xgb = 1-(1-r2_xgb)*(len(df)-1)/(len(df)-len(df.columns)-1)

# TREINAMENTO DO MODELO RANDOM FOREST
rf = RandomForestRegressor(n_estimators= 600, max_depth=25, random_state=200).fit(X_train,y_train)

# PREDIÇÃO DO MODELO RANDOM FOREST
yrf = rf.predict(X_test)

# MÉTRICAS DO MODELO RANDOM FOREST
mae_rf = mean_absolute_error(y_test,yrf)
mse_rf = mean_squared_error(y_test,yrf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test,yrf)
adj_r2_rf = 1-(1-r2_rf)*(len(df)-1)/(len(df)-len(df.columns)-1)

# PRINT DAS MÉTRICAS ALCANÇADAS POR CADA MODELO
print('--------MÉTRICAS XGBOOST--------')
print(f'mae: {mae_xgb}\nmse: {mse_xgb}\nrmse: {rmse_xgb}\nr2: {r2_xgb}\nr2 ajustado: {adj_r2_xgb}')
print('')
print('-----MÉTRICAS RANDOM FOREST-----')
print(f'mae: {mae_rf}\nmse: {mse_rf}\nrmse: {rmse_rf}\nr2: {r2_rf}\nr2 ajustado: {adj_r2_rf}')

--------MÉTRICAS XGBOOST--------
mae: 6938.246973876953
mse: 132015382.57326475
rmse: 11489.794714148062
r2: 0.3843833454171841
r2 ajustado: 0.35294760135338077

-----MÉTRICAS RANDOM FOREST-----
mae: 6899.028977777778
mse: 88527321.40201999
rmse: 9408.895865191622
r2: 0.5871777032464838
r2 ajustado: 0.5660974157526872


In [41]:
# TIRANDO A MÉDIA DAS PREDIÇÕES DE CADA UM DOS MODELOS PARA VER SE O RESULTADO MELHORA (TEOREMA DAS MULTIDÕES)
ymisto = (yrf + yxgb)/2

mae_misto = mean_absolute_error(y_test,ymisto)
mse_misto = mean_squared_error(y_test,ymisto)
rmse_misto = np.sqrt(mse_misto)
r2_misto = r2_score(y_test,ymisto)
adj_r2_misto = 1-(1-r2_misto)*(len(df)-1)/(len(df)-len(df.columns)-1)

print('-----MÉTRICAS MISTO-----')
print(f'mae: {mae_misto}\nmse: {mse_misto}\nrmse: {rmse_misto}\nr2: {r2_misto}\nr2 ajustado: {adj_r2_misto}')


-----MÉTRICAS MISTO-----
mae: 6775.626248787435
mse: 102557709.90240268
rmse: 10127.078053535613
r2: 0.5217509274970038
r2 ajustado: 0.49732969826280826


#### OTIMIZAÇÃO DOS MODELOS

> GRIDSEARCHCV - XGBOOST

In [42]:
# Definição dos parâmetros a serem testados

parameters = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

# Inicializando o modelo

xgb_model = XGBRegressor()

# Encontrando os melhores hiperparâmetros 

grid_search = GridSearchCV(estimator=xgb_model, param_grid=parameters, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Extraindo a melhor combinação de parâmetros encontrada

best_params = grid_search.best_params_
print("Melhores hiperparâmetros:", best_params)
print('')

# Treinando o modelo com os melhores hiperparâmetros

grid_xgb = XGBRegressor(**best_params)
grid_xgb.fit(X_train, y_train)

# Previsão do modelo otimizado

y_grid_xgb = grid_xgb.predict(X_test)

# Avaliando os resultados

mae_grid_xgb = mean_absolute_error(y_test,y_grid_xgb)
mse_grid_xgb = mean_squared_error(y_test,y_grid_xgb)
rmse_grid_xgb = np.sqrt(mse_grid_xgb)
r2_grid_xgb = r2_score(y_test,y_grid_xgb)
adj_r2_grid_xgb = 1-(1-r2_grid_xgb)*(len(df)-1)/(len(df)-len(df.columns)-1)

print('--------MÉTRICAS XGBOOST OTIMIZADO--------')
print(f'mae: {mae_grid_xgb}\nmse: {mse_grid_xgb}\nrmse: {rmse_grid_xgb}\nr2: {r2_grid_xgb}\nr2 ajustado: {adj_r2_grid_xgb}')

Melhores hiperparâmetros: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9}

--------MÉTRICAS XGBOOST OTIMIZADO--------
mae: 6976.92524576823
mse: 82888896.72103375
rmse: 9104.333952631228
r2: 0.6134709129586139
r2 ajustado: 0.5937332574501175


>BAYESSEARCHCV - XGBOOST

In [43]:
# Inicializando o modelo e definindo os parâmetros a serem testados
opt_xgb = BayesSearchCV(
                    XGBRegressor(),
                    {
                    'learning_rate': [0.01, 0.1, 0.2],
                    'n_estimators': [100, 200, 300],
                    'max_depth': [3, 4, 5],
                    'min_child_weight': [1, 2, 3],
                    'subsample': [0.8, 0.9, 1.0],
                    'colsample_bytree': [0.8, 0.9, 1.0],
                    },
                    scoring='neg_mean_squared_error',
                    cv=5
                   )

opt_xgb.fit(X_train, y_train)

# Extraindo a melhor combinação de parâmetros encontrada
best_params_bayes_xgb = opt_xgb.best_params_

# Criação dos modelos com os melhores parâmetros encontrados
bayes_xgb = XGBRegressor(**best_params_bayes_xgb)

# Treinamento do modelo otimizado
bayes_xgb.fit(X_train, y_train)

# Previsões do modelo otimizado
y_bayes_xgb = bayes_xgb.predict(X_test)

# Avaliando os resultados

mae_bayes_xgb = mean_absolute_error(y_test,y_bayes_xgb)
mse_bayes_xgb = mean_squared_error(y_test,y_bayes_xgb)
rmse_bayes_xgb = np.sqrt(mse_bayes_xgb)
r2_bayes_xgb = r2_score(y_test,y_bayes_xgb)
adj_r2_bayes_xgb = 1-(1-r2_bayes_xgb)*(len(df)-1)/(len(df)-len(df.columns)-1)

print('--------MÉTRICAS XGBOOST OTIMIZADO--------')
print(f'mae: {mae_bayes_xgb}\nmse: {mse_bayes_xgb}\nrmse: {rmse_bayes_xgb}\nr2: {r2_bayes_xgb}\nr2 ajustado: {adj_r2_bayes_xgb}')

--------MÉTRICAS XGBOOST OTIMIZADO--------
mae: 6976.92524576823
mse: 82888896.72103375
rmse: 9104.333952631228
r2: 0.6134709129586139
r2 ajustado: 0.5937332574501175


> GRIDSEARCHCV - RANDOM FOREST

In [44]:
# Definição dos parâmetros a serem testados

parameters = {
    'n_estimators': [500,550,600,650,700],
    'max_depth': [20,25,30,35],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'random_state':[200]
}

# Inicializando o modelo

rf_model = RandomForestRegressor()

# Encontrando os melhores hiperparâmetros 

grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=parameters, scoring='neg_mean_squared_error', cv=5)
grid_search_rf.fit(X_train, y_train)

# Extraindo a melhor combinação de parâmetros encontrada

best_params_rf = grid_search_rf.best_params_
print("Melhores hiperparâmetros:", best_params_rf)
print('')

# Treinando o modelo com os melhores hiperparâmetros

grid_rf = RandomForestRegressor(**best_params_rf)
grid_rf.fit(X_train, y_train)

# Previsão do modelo otimizado

y_grid_rf = grid_rf.predict(X_test)

# Avaliando os resultados

mae_grid_rf = mean_absolute_error(y_test,y_grid_rf)
mse_grid_rf = mean_squared_error(y_test,y_grid_rf)
rmse_grid_rf = np.sqrt(mse_grid_rf)
r2_grid_rf = r2_score(y_test,y_grid_rf)
adj_r2_grid_rf = 1-(1-r2_grid_rf)*(len(df)-1)/(len(df)-len(df.columns)-1)

print('--------MÉTRICAS RANDOM FOREST OTIMIZADO--------')
print(f'mae: {mae_grid_rf}\nmse: {mse_grid_rf}\nrmse: {rmse_grid_rf}\nr2: {r2_grid_rf}\nr2 ajustado: {adj_r2_grid_rf}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Dudu_\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Dudu_\AppData\Local\Temp\ipykernel_11872\1974531839.py", line 20, in <module>
    grid_search_rf.fit(X_train, y_train)
  File "c:\Users\Dudu_\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dudu_\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py", line 898, in fit
    self._run_search(evaluate_candidates)
  File "c:\Users\Dudu_\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py", line 1419, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "c:\Users\Dudu_\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py", line 845, in evaluate_candidates
    out = parallel(
          ^^^^^^^

> BAYESSEARCHCV - RANDOM FOREST

In [ ]:
# Inicializando o modelo e definindo os parâmetros a serem testados
opt_rf = BayesSearchCV(
                    RandomForestRegressor(),
                    {
                     'n_estimators': [500,550,600,650,700],
                     'max_depth': [20,25,30,35],
                     'min_samples_split': [2, 5, 10],
                     'min_samples_leaf': [1, 2, 4],
                     'max_features': ['sqrt', 'log2'],
                     'bootstrap': [True, False],
                     'random_state':[200]
                    },
                    scoring='neg_mean_squared_error',
                    cv=5
                   )

opt_rf.fit(X_train, y_train)

# Extraindo a melhor combinação de parâmetros encontrada
best_params_bayes_rf = opt_rf.best_params_

# Criação dos modelos com os melhores parâmetros encontrados
bayes_rf = XGBRegressor(**best_params_bayes_rf)

# Treinamento do modelo otimizado
bayes_rf.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_bayes_rf = bayes_rf.predict(X_test)

# Avaliando os resultados

mae_bayes_rf = mean_absolute_error(y_test,y_bayes_rf)
mse_bayes_rf = mean_squared_error(y_test,y_bayes_rf)
rmse_bayes_rf = np.sqrt(mse_bayes_rf)
r2_bayes_rf = r2_score(y_test,y_bayes_rf)
adj_r2_bayes_rf = 1-(1-r2_bayes_rf)*(len(df)-1)/(len(df)-len(df.columns)-1)

print('--------MÉTRICAS RANDOM FOREST OTIMIZADO--------')
print(f'mae: {mae_bayes_rf}\nmse: {mse_bayes_rf}\nrmse: {rmse_bayes_rf}\nr2: {r2_bayes_rf}\nr2 ajustado: {adj_r2_bayes_rf}')

--------MÉTRICAS RANDOM FOREST OTIMIZADO--------
mae: 5075.5397716446505
mse: 44370049.305884816
rmse: 6661.08469439361
r2: 0.7657346449139918
r2 ajustado: 0.7536174713750603


In [ ]:
# TIRANDO A MÉDIA DAS PREDIÇÕES DE CADA UM DOS MODELOS PARA VER SE O RESULTADO MELHORA
ymisto2 = (yrf + y_grid_xgb)/2

mae_misto2 = mean_absolute_error(y_test,ymisto2)
mse_misto2 = mean_squared_error(y_test,ymisto2)
rmse_misto2 = np.sqrt(mse_misto2)
r2_misto2 = r2_score(y_test,ymisto2)
adj_r2_misto2 = 1-(1-r2_misto2)*(len(df)-1)/(len(df)-len(df.columns)-1)

print('-----MÉTRICAS MISTO-----')
print(f'mae: {mae_misto2}\nmse: {mse_misto2}\nrmse: {rmse_misto2}\nr2: {r2_misto2}\nr2 ajustado: {adj_r2_misto2}')



-----MÉTRICAS MISTO-----
mae: 4726.665250154786
mse: 36144667.12717195
rmse: 6012.04350675974
r2: 0.8091630861025556
r2 ajustado: 0.7992922112457912


>> Mesmo com a otimização feita utilizando o GridSearch e o a busca Bayesiana, os resultados encontrados antes da tentativa de otimização foram melhores. Dessa forma, foi decidido manter o modelo da forma inicial.

### TESTE FINAL

#### PREDIÇÃO JOGO DO VASCO - DIA 22/11

O jogo do Cruzeiro x Vasco, do dia 22/11, não contou com a presença de torcedores no estádio por conta de uma punição sofrida pelo Cruzeiro (punição decorrente da invasão de torcedores cruzeirenses no jogo da 34ª rodada do Campeonato Brasileiro contra o Coritiba, na Vila Capanema).
Por conta disso, foi decidido fazer a previsão com o modelo criado da possível quantidade de público nesse jogo caso a punição não existisse.

In [ ]:
# PREDIÇÃO DE PÚBLICO DO JOGO CRUZEIRO X VASCO

# DEFININDO OS ATRIBUTOS
vasco = np.array([2023,1900,11,0.491120,1,0,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False])

# REDIMENSIONANDO A ARRAY PARA FICAR NO FORMATO IDEAL DE ENTRADA NO MODELO
vasco = vasco.reshape(1,-1)

# PREDIÇÃO DOS DOIS MODELOS 
yvasco_xgb = xgb.predict(vasco)
yvasco_rf = rf.predict(vasco)

# MÉDIA DA AVALIAÇÃO DOS DOIS MODELOS PARA ENCONTRAR O RESULTADO FINAL
publico_possivel = (yvasco_xgb + yvasco_rf)/2

print(f"A previsão de público para o jogo entre Cruzeiro e Vasco foi de {publico_possivel[0]:.0f} torcedores.")

A previsão de público para o jogo entre Cruzeiro e Vasco foi de 34337 torcedores.
